In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/grid3_u_27000_1/"

"../../graphs/grid3_u_27000_1/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:5
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

@time for i in 4:5
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.758286 seconds (535.44 k allocations: 29.664 MB, 2.87% gc time)
  0.418711 seconds (539.75 k allocations: 12.914 MB)
  8.595714 seconds (86.77 M allocations: 2.193 GB, 4.54% gc time)
Working on iteration 10 with a-norm error 8253.893714510117
Working on iteration 20 with a-norm error 7322.756364124031
Working on iteration 30 with a-norm error 6215.650957695415
Working on iteration 40 with a-norm error 5265.880738436075
Working on iteration 50 with a-norm error 4409.532748804799
Working on iteration 60 with a-norm error 3558.3407480737174
Working on iteration 70 with a-norm error 2795.472698245343
Working on iteration 80 with a-norm error 2296.230860415408
Working on iteration 90 with a-norm error 1923.8042880423288
Working on iteration 100 with a-norm error 1481.6012499928665
Working on iteration 110 with a-norm error 1233.3957483852564
Working on iteration 120 with a-norm error 988.4496467190421
Working on iteration 130 with a-norm error 813.9542998075443
Working on iteration 140 